In [8]:
val obamaSpeechRdd = sc.textFile("data/state-of-union")

obamaSpeechRdd: org.apache.spark.rdd.RDD[String] = data/state-of-union MapPartitionsRDD[1] at textFile at <console>:25


Convert to lowercase

In [13]:
val obamaSpeechLowerRdd = obamaSpeechRdd.map(_.toLowerCase())

obamaSpeechLowerRdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[3] at map at <console>:26


Split line into a list of words

In [133]:
//val obamaWordsRdd = obamaSpeechLowerRdd.flatMap(x => x.split("\\s+")).map(_.replaceAll("\\W","")).filter(!_.equals(""))

val obamaWordsRdd = obamaSpeechLowerRdd.flatMap(x => x.split("\\W+"))
obamaWordsRdd.cache()

obamaWordsRdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[359] at flatMap at <console>:30
res68: obamaWordsRdd.type = MapPartitionsRDD[359] at flatMap at <console>:30


Remove stop words from the state of union speeches

In [134]:
val stopWordsRdd = sc.textFile("data/stop-words/stop-word-list.txt")
val obamaNoStopRdd = obamaWordsRdd.subtract(stopWordsRdd)

stopWordsRdd: org.apache.spark.rdd.RDD[String] = data/stop-words/stop-word-list.txt MapPartitionsRDD[361] at textFile at <console>:29
obamaNoStopRdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[365] at subtract at <console>:30


Count

In [135]:
val obamaWordCoundRdd = obamaNoStopRdd.map((_,1)).reduceByKey(_+_)
//obamaWordCoundRdd.coalesce(1).sortByKey().foreach(println)
obamaWordCoundRdd.coalesce(1).sortBy(x => x._2,false).take(20).foreach(println)
//obamaWordCoundRdd.foreach(println)
//obamaWordCoundRdd.takeOrdered(20)(Ordering[Int].reverse.on(_._2)).foreach(println)
/*
(s,537)
(t,225)
(america,199)
(new,197)
(people,176)
(jobs,167)
(american,154)
*/

(s,537)
(t,225)
(america,199)
(new,197)
(people,176)
(jobs,167)
(american,154)
(years,143)
(americans,131)
(work,131)
(make,128)
(year,127)
(time,117)
(let,117)
(ve,116)
(know,106)
(country,105)
(like,104)
(world,103)
(economy,98)


obamaWordCoundRdd: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[367] at reduceByKey at <console>:28


Count the number of words that start with each letter in alphabet

What is the distribution like?

Instead of creating a tuple of (word, 1), create (word(0), 1) or (word.charAt(0), 1)



In [141]:
val startsWithRdd = obamaWordsRdd.filter(!_.equals(""))
    .map(x=>(x.charAt(0),1))
    .reduceByKey((x,y)=>x+y)
    .sortByKey()
startsWithRdd.collect().foreach(println)

(0,38)
(1,82)
(2,64)
(3,20)
(4,11)
(5,20)
(6,12)
(7,11)
(8,6)
(9,16)
(a,5508)
(b,2078)
(c,2483)
(d,1426)
(e,1305)
(f,1837)
(g,845)
(h,1663)
(i,3019)
(j,390)
(k,294)
(l,1160)
(m,1763)
(n,1243)
(o,3332)
(p,1707)
(q,72)
(r,1364)
(s,3273)
(t,7927)
(u,551)
(v,315)
(w,3724)
(y,620)
(z,2)


startsWithRdd: org.apache.spark.rdd.RDD[(Char, Int)] = ShuffledRDD[388] at sortByKey at <console>:31


What are some of the longest words in all the speeches?

In [145]:
val wordLengthRdd = obamaWordsRdd.map(x => (x.length(),x))
wordLengthRdd.sortByKey().collect().foreach(println)

2019-04-23 21:40:33 WARN  TransportChannelHandler:78 - Exception in connection from /10.20.8.238:55590
java.io.IOException: Operation timed out
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:380)
	at io.netty.buffer.PooledUnsafeDirectByteBuf.setBytes(PooledUnsafeDirectByteBuf.java:288)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1106)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:343)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:123)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:645)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:580)
	at io.netty.channel.nio.NioEventLoop.process

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 283.0 failed 1 times, most recent failure: Lost task 0.0 in stage 283.0 (TID 1042, localhost, executor driver): java.lang.ClassCastException: cannot assign instance of scala.collection.immutable.List$SerializationProxy to field org.apache.spark.rdd.RDD.org$apache$spark$rdd$RDD$$dependencies_ of type scala.collection.Seq in instance of org.apache.spark.rdd.MapPartitionsRDD

2019-04-23 21:41:48 ERROR Utils:91 - Aborting task
java.io.IOException: Failed to connect to /10.20.8.238:55590
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:245)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:187)
	at org.apache.spark.rpc.netty.NettyRpcEnv.org$apache$spark$rpc$netty$NettyRpcEnv$$downloadClient(NettyRpcEnv.scala:368)
	at org.apache.spark.rpc.netty.NettyRpcEnv$$anonfun$openChannel$1.apply$mcV$sp(NettyRpcEnv.scala:336)
	at org.apache.spark.rpc.netty.NettyRpcEnv$$anonfun$openChannel$1.apply(NettyRpcEnv.scala:335)
	at org.apache.spark.rpc.netty.NettyRpcEnv$$anonfun$openChannel$1.apply(NettyRpcEnv.scala:335)
	at org.apache.spark.util.Utils$.tryWithSafeFinallyAndFailureCallbacks(Utils.scala:1394)
	at org.apache.spark.rpc.netty.NettyRpcEnv.openChannel(NettyRpcEnv.scala:339)
	at org.apache.spark.repl.ExecutorClassLoader.org$apache$spark$repl$ExecutorClassLoader$$getClassF

2019-04-23 21:41:48 WARN  TaskSetManager:66 - Lost task 3.0 in stage 283.0 (TID 1045, localhost, executor driver): TaskKilled (Stage cancelled)
2019-04-23 21:43:06 ERROR Utils:91 - Aborting task
java.io.IOException: Failed to connect to /10.20.8.238:55590
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:245)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:187)
	at org.apache.spark.rpc.netty.NettyRpcEnv.org$apache$spark$rpc$netty$NettyRpcEnv$$downloadClient(NettyRpcEnv.scala:368)
	at org.apache.spark.rpc.netty.NettyRpcEnv$$anonfun$openChannel$1.apply$mcV$sp(NettyRpcEnv.scala:336)
	at org.apache.spark.rpc.netty.NettyRpcEnv$$anonfun$openChannel$1.apply(NettyRpcEnv.scala:335)
	at org.apache.spark.rpc.netty.NettyRpcEnv$$anonfun$openChannel$1.apply(NettyRpcEnv.scala:335)
	at org.apache.spark.util.Utils$.tryWithSafeFinallyAndFailureCallbacks(Utils.scala:1394)
	at org.apache.spark.rpc.netty.N

2019-04-23 21:43:07 WARN  TaskSetManager:66 - Lost task 2.0 in stage 283.0 (TID 1044, localhost, executor driver): TaskKilled (Stage cancelled)
